In [338]:
%reset -fs
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import pickle
import re

<div class="alert alert-block alert-danger">

 <span style="color:red ; font-size:1.2em" > NOTE TO CODE REVIEWER: At 11am on Feb 22, my access to the web site I used to scrape my data was blocked. The code in this notebook is fine since it uses pickled files created before the block, unless the other notebooks in my submission were run and the pickled files they generate altered.</span>
</div>

<div class="alert alert-block alert-info">

</div>

<div class="alert alert-block alert-info">
<h3> In this notebook:</h3>
<br>
<span style="font-size:1.2em" >
    <ul>
        <li>I combine the data from the dataframes in the previous steps into a comprehensive dataframe for 2019 and reorganize columns</li>
        <li> All data elements not found during scraping and labeled 'nf' are replaced with Nan</li>
        <li>A final dataframe for 2019 is created and pickled</li>
    </ul>
</span>
</div>

<div class="alert alert-block alert-info">

</div>

In [339]:
# Create a final dataframe for 2019 and read in main movie list data frame

movies_final_2019_df = pd.read_pickle('df_all_movies_2019_pk')
movies_final_2019_df.head(10)

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url
0,1,Avengers: Endgame,"Apr 26, 2019",Walt Disney,Action,"$858,373,000","93,708,843",https://www.the-numbers.com/movie/Avengers-End...,https://www.the-numbers.com/movie/Avengers-End...
1,2,The Lion King,"Jul 19, 2019",Walt Disney,Adventure,"$543,638,043","59,349,131",https://www.the-numbers.com/movie/Lion-King-Th...,https://www.the-numbers.com/movie/Lion-King-Th...
2,3,Frozen II,"Nov 22, 2019",Walt Disney,Adventure,"$450,439,533","49,174,621",https://www.the-numbers.com/movie/Frozen-II-(2...,https://www.the-numbers.com/movie/Frozen-II-(2...
3,4,Toy Story 4,"Jun 21, 2019",Walt Disney,Adventure,"$434,038,008","47,384,062",https://www.the-numbers.com/movie/Toy-Story-4-...,https://www.the-numbers.com/movie/Toy-Story-4-...
4,5,Captain Marvel,"Mar 8, 2019",Walt Disney,Action,"$426,829,839","46,597,144",https://www.the-numbers.com/movie/Captain-Marv...,https://www.the-numbers.com/movie/Captain-Marv...
5,6,Star Wars: The Rise of Skyw…,"Dec 20, 2019",Walt Disney,Adventure,"$390,706,234","42,653,519",https://www.the-numbers.com/movie/Star-Wars-Th...,https://www.the-numbers.com/movie/Star-Wars-Th...
6,7,Spider-Man: Far From Home,"Jul 2, 2019",Sony Pictures,Action,"$390,532,085","42,634,507",https://www.the-numbers.com/movie/Spider-Man-F...,https://www.the-numbers.com/movie/Spider-Man-F...
7,8,Aladdin,"May 24, 2019",Walt Disney,Adventure,"$355,559,216","38,816,508",https://www.the-numbers.com/movie/Aladdin-(201...,https://www.the-numbers.com/movie/Aladdin-(201...
8,9,Joker,"Oct 4, 2019",Warner Bros.,Thriller/Suspense,"$333,772,511","36,438,047",https://www.the-numbers.com/movie/Joker-(2019)...,https://www.the-numbers.com/movie/Joker-(2019)...
9,10,It: Chapter Two,"Sep 6, 2019",Warner Bros.,Horror,"$211,593,228","23,099,697",https://www.the-numbers.com/movie/It-Chapter-T...,https://www.the-numbers.com/movie/It-Chapter-T...


In [340]:
#read Legs dataframe

legs_2019_df = pd.read_pickle('legs_2019_df_pk')
legs_2019_df['Legs']

0       2.40
1       2.83
2       3.66
3       3.59
4       2.78
       ...  
675     2.29
676     4.17
677     2.04
678     6.96
679    18.30
Name: Legs, Length: 680, dtype: float64

In [341]:
#Add the Legs dataframe as column

movies_final_2019_df['Legs'] = legs_2019_df['Legs']

In [342]:
movies_final_2019_df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url,Legs
0,1,Avengers: Endgame,"Apr 26, 2019",Walt Disney,Action,"$858,373,000","93,708,843",https://www.the-numbers.com/movie/Avengers-End...,https://www.the-numbers.com/movie/Avengers-End...,2.40
1,2,The Lion King,"Jul 19, 2019",Walt Disney,Adventure,"$543,638,043","59,349,131",https://www.the-numbers.com/movie/Lion-King-Th...,https://www.the-numbers.com/movie/Lion-King-Th...,2.83
2,3,Frozen II,"Nov 22, 2019",Walt Disney,Adventure,"$450,439,533","49,174,621",https://www.the-numbers.com/movie/Frozen-II-(2...,https://www.the-numbers.com/movie/Frozen-II-(2...,3.66
3,4,Toy Story 4,"Jun 21, 2019",Walt Disney,Adventure,"$434,038,008","47,384,062",https://www.the-numbers.com/movie/Toy-Story-4-...,https://www.the-numbers.com/movie/Toy-Story-4-...,3.59
4,5,Captain Marvel,"Mar 8, 2019",Walt Disney,Action,"$426,829,839","46,597,144",https://www.the-numbers.com/movie/Captain-Marv...,https://www.the-numbers.com/movie/Captain-Marv...,2.78
...,...,...,...,...,...,...,...,...,...,...
675,676,Chef Flynn,"Nov 9, 2018",Kino Lorber,Documentary,$721,78,https://www.the-numbers.com/movie/Chef-Flynn-(...,https://www.the-numbers.com/movie/Chef-Flynn-(...,2.29
676,677,I am Not a Witch,"Sep 7, 2018",Film Movement,Drama,$550,60,https://www.the-numbers.com/movie/I-am-Not-a-W...,https://www.the-numbers.com/movie/I-am-Not-a-W...,4.17
677,678,Divide and Conquer: The Sto…,"Dec 7, 2018",Magnolia Pictures,Documentary,$474,51,https://www.the-numbers.com/movie/Divide-and-C...,https://www.the-numbers.com/movie/Divide-and-C...,2.04
678,679,Senso,"Jul 8, 1968",Fleetwood Films,Drama,$437,47,https://www.the-numbers.com/movie/Senso-(Italy...,https://www.the-numbers.com/movie/Senso-(Italy...,6.96


In [343]:
#Read budget dataframe and add to final dataframe

budget_2019_df = pd.read_pickle('budget_2019_pk')
wbo_to_budget_ratio_2019_df = pd.read_pickle('wbo_to_budget_ratio_2019_pk')

movies_final_2019_df['Budget'] = budget_2019_df['Production_Budget']
movies_final_2019_df['WBO_to_Budget_Ratio'] = (
    wbo_to_budget_ratio_2019_df['WBO to Budget Ratio'])

In [344]:
movies_final_2019_df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url,Legs,Budget,WBO_to_Budget_Ratio
0,1,Avengers: Endgame,"Apr 26, 2019",Walt Disney,Action,"$858,373,000","93,708,843",https://www.the-numbers.com/movie/Avengers-End...,https://www.the-numbers.com/movie/Avengers-End...,2.40,400000000,7.0
1,2,The Lion King,"Jul 19, 2019",Walt Disney,Adventure,"$543,638,043","59,349,131",https://www.the-numbers.com/movie/Lion-King-Th...,https://www.the-numbers.com/movie/Lion-King-Th...,2.83,260000000,6.4
2,3,Frozen II,"Nov 22, 2019",Walt Disney,Adventure,"$450,439,533","49,174,621",https://www.the-numbers.com/movie/Frozen-II-(2...,https://www.the-numbers.com/movie/Frozen-II-(2...,3.66,150000000,9.6
3,4,Toy Story 4,"Jun 21, 2019",Walt Disney,Adventure,"$434,038,008","47,384,062",https://www.the-numbers.com/movie/Toy-Story-4-...,https://www.the-numbers.com/movie/Toy-Story-4-...,3.59,200000000,5.4
4,5,Captain Marvel,"Mar 8, 2019",Walt Disney,Action,"$426,829,839","46,597,144",https://www.the-numbers.com/movie/Captain-Marv...,https://www.the-numbers.com/movie/Captain-Marv...,2.78,175000000,6.5
...,...,...,...,...,...,...,...,...,...,...,...,...
675,676,Chef Flynn,"Nov 9, 2018",Kino Lorber,Documentary,$721,78,https://www.the-numbers.com/movie/Chef-Flynn-(...,https://www.the-numbers.com/movie/Chef-Flynn-(...,2.29,nf,nf
676,677,I am Not a Witch,"Sep 7, 2018",Film Movement,Drama,$550,60,https://www.the-numbers.com/movie/I-am-Not-a-W...,https://www.the-numbers.com/movie/I-am-Not-a-W...,4.17,nf,nf
677,678,Divide and Conquer: The Sto…,"Dec 7, 2018",Magnolia Pictures,Documentary,$474,51,https://www.the-numbers.com/movie/Divide-and-C...,https://www.the-numbers.com/movie/Divide-and-C...,2.04,nf,nf
678,679,Senso,"Jul 8, 1968",Fleetwood Films,Drama,$437,47,https://www.the-numbers.com/movie/Senso-(Italy...,https://www.the-numbers.com/movie/Senso-(Italy...,6.96,nf,nf


In [345]:
#Read theater count and average run dataframe and add to final dataframe

theater_count_2019_df = pd.read_pickle('theater_count_2019_pk')
avg_run_2019_df = pd.read_pickle('avg_run_2019_pk')

movies_final_2019_df['Theater_Count'] = theater_count_2019_df['Theater_Count']
movies_final_2019_df['Average_Run'] = avg_run_2019_df['Average_Run']


In [346]:
#Read source dataframe and add to final dataframe

source_2019_df = pd.read_pickle('source_2019_pk')
movies_final_2019_df['Source'] = source_2019_df['Source']
# movies_final_2019_df

In [347]:
#Read creative type dataframe and add to final dataframe

creative_type_2019_df =pd.read_pickle('creative_type_2019_pk') 
movies_final_2019_df['Creative_Type'] = creative_type_2019_df['Creative_Type']
movies_final_2019_df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url,Legs,Budget,WBO_to_Budget_Ratio,Theater_Count,Average_Run,Source,Creative_Type
0,1,Avengers: Endgame,"Apr 26, 2019",Walt Disney,Action,"$858,373,000","93,708,843",https://www.the-numbers.com/movie/Avengers-End...,https://www.the-numbers.com/movie/Avengers-End...,2.40,400000000,7.0,4662,8.1,Based on Comic/Graphic Novel,Super Hero
1,2,The Lion King,"Jul 19, 2019",Walt Disney,Adventure,"$543,638,043","59,349,131",https://www.the-numbers.com/movie/Lion-King-Th...,https://www.the-numbers.com/movie/Lion-King-Th...,2.83,260000000,6.4,4725,8.5,Remake,Kids Fiction
2,3,Frozen II,"Nov 22, 2019",Walt Disney,Adventure,"$450,439,533","49,174,621",https://www.the-numbers.com/movie/Frozen-II-(2...,https://www.the-numbers.com/movie/Frozen-II-(2...,3.66,150000000,9.6,4440,8.6,Based on Folk Tale/Legend/Fairytale,Kids Fiction
3,4,Toy Story 4,"Jun 21, 2019",Walt Disney,Adventure,"$434,038,008","47,384,062",https://www.the-numbers.com/movie/Toy-Story-4-...,https://www.the-numbers.com/movie/Toy-Story-4-...,3.59,200000000,5.4,4575,8.3,Original Screenplay,Kids Fiction
4,5,Captain Marvel,"Mar 8, 2019",Walt Disney,Action,"$426,829,839","46,597,144",https://www.the-numbers.com/movie/Captain-Marv...,https://www.the-numbers.com/movie/Captain-Marv...,2.78,175000000,6.5,4310,7.9,Based on Comic/Graphic Novel,Super Hero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,676,Chef Flynn,"Nov 9, 2018",Kino Lorber,Documentary,$721,78,https://www.the-numbers.com/movie/Chef-Flynn-(...,https://www.the-numbers.com/movie/Chef-Flynn-(...,2.29,nf,nf,1,3.4,Based on Real Life Events,Factual
676,677,I am Not a Witch,"Sep 7, 2018",Film Movement,Drama,$550,60,https://www.the-numbers.com/movie/I-am-Not-a-W...,https://www.the-numbers.com/movie/I-am-Not-a-W...,4.17,nf,nf,2,6.0,Original Screenplay,Contemporary Fiction
677,678,Divide and Conquer: The Sto…,"Dec 7, 2018",Magnolia Pictures,Documentary,$474,51,https://www.the-numbers.com/movie/Divide-and-C...,https://www.the-numbers.com/movie/Divide-and-C...,2.04,nf,nf,15,1.5,Based on Real Life Events,Factual
678,679,Senso,"Jul 8, 1968",Fleetwood Films,Drama,$437,47,https://www.the-numbers.com/movie/Senso-(Italy...,https://www.the-numbers.com/movie/Senso-(Italy...,6.96,nf,nf,nf,nf,Based on Fiction Book/Short Story,Historical Fiction


In [348]:
#Drop url address columns which are no longer needed

movies_final_2019_df.drop(['summary_page_url', 'cast_page_url'],
                          axis = 1, inplace = True)
movies_final_2019_df.shape

(680, 16)

In [349]:
#Re-order columns

col_reorder = ['Rank', 'Movie','ReleaseDate', 'Revenues', 'Tickets_Sold',
              'Legs', 'Budget', 'WBO_to_Budget_Ratio', 'Theater_Count',
               'Average_Run', 'Genre', 'Source', 'Creative_Type', 'Distributor']

movies_final_2019_df = movies_final_2019_df[col_reorder]

In [350]:
# movies_final_2019_df

In [351]:
#Read cast dataframe and join final dataframe

cast_2019_df = pd.read_pickle('cast_2019_pk')
# cast_2019_df

In [352]:
movies_final_2019_df = movies_final_2019_df.join(cast_2019_df)

In [2]:
#Interim save of file to pickle

# movies_final_2019_df.to_pickle('final_2019_pk')

In [354]:
#Create a dictionary to store which columns have how many not found 'nf' elements

nf_count_dict = {}


for name in col_reorder:
    count_nf = movies_final_2019_df[movies_final_2019_df[name] == 'nf'][name].count()
    print(f'{name}   {count_nf}')


    
    

Rank   0
Movie   0
ReleaseDate   0
Revenues   0
Tickets_Sold   0
Legs   0
Budget   485
WBO_to_Budget_Ratio   485
Theater_Count   37
Average_Run   37
Genre   0
Source   19
Creative_Type   13
Distributor   0


In [356]:
#A check of the final dataframe data types

movies_final_2019_df.dtypes
# movies_final_2019_df

Rank                    object
Movie                   object
ReleaseDate             object
Revenues                object
Tickets_Sold            object
Legs                   float64
Budget                  object
WBO_to_Budget_Ratio     object
Theater_Count           object
Average_Run             object
Genre                   object
Source                  object
Creative_Type           object
Distributor             object
All_Cast_Rating        float64
Samuel L. Jackson        int64
Robert Downey, Jr.       int64
Scarlett Johansson       int64
Tom Hanks                int64
Bradley Cooper           int64
Harrison Ford            int64
Chris Evans              int64
Tom Cruise               int64
Chris Hemsworth          int64
Zoe Saldana              int64
dtype: object

In [361]:
#Replace all not found 'nf' elements with Nan
#Replace string numeric columns with numbers after stripping $ and ,


movies_final_2019_df.replace(r'nf', np.nan, regex = True, inplace = True )


col_list = ['Revenues', 'Tickets_Sold', 'Legs', 'Budget', 'WBO_to_Budget_Ratio',
           'Theater_Count', 'Average_Run']

for col_name in col_list:
    movies_final_2019_df[col_name] = (movies_final_2019_df[col_name]
                                    .replace('[\$,]', '', regex = True))
                                        
    movies_final_2019_df[col_name] = (pd.to_numeric(movies_final_2019_df[col_name], 
                                                errors = 'coerce'))
    


In [362]:
#another check of dataframe data types

movies_final_2019_df.dtypes
# movies_final_2019_df

,Rank,Movie,ReleaseDate,Revenues,Tickets_Sold,Legs,Budget,WBO_to_Budget_Ratio,Theater_Count,Average_Run,...,Samuel L. Jackson,"Robert Downey, Jr.",Scarlett Johansson,Tom Hanks,Bradley Cooper,Harrison Ford,Chris Evans,Tom Cruise,Chris Hemsworth,Zoe Saldana
0,1,Avengers: Endgame,"Apr 26, 2019",858373000,93708843,2.40,400000000.0,7.0,4662.0,8.1,...,1,1,1,0,1,0,1,0,1,1
1,2,The Lion King,"Jul 19, 2019",543638043,59349131,2.83,260000000.0,6.4,4725.0,8.5,...,0,0,0,0,0,0,0,0,0,0
2,3,Frozen II,"Nov 22, 2019",450439533,49174621,3.66,150000000.0,9.6,4440.0,8.6,...,0,0,0,0,0,0,0,0,0,0
3,4,Toy Story 4,"Jun 21, 2019",434038008,47384062,3.59,200000000.0,5.4,4575.0,8.3,...,0,0,0,1,0,0,0,0,0,0
4,5,Captain Marvel,"Mar 8, 2019",426829839,46597144,2.78,175000000.0,6.5,4310.0,7.9,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,676,Chef Flynn,"Nov 9, 2018",721,78,2.29,NaN,NaN,1.0,3.4,...,0,0,0,0,0,0,0,0,0,0
676,677,I am Not a Witch,"Sep 7, 2018",550,60,4.17,NaN,NaN,2.0,6.0,...,0,0,0,0,0,0,0,0,0,0
677,678,Divide and Conquer: The Sto…,"Dec 7, 2018",474,51,2.04,NaN,NaN,15.0,1.5,...,0,0,0,0,0,0,0,0,0,0
678,679,Senso,"Jul 8, 1968",437,47,6.96,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [363]:
#Another check to make sure there are no more not found 'nf' values

for name in col_reorder:
    count_nf = movies_final_2019_df[movies_final_2019_df[name] == 'nf'][name].count()
    print(f'{name}   {count_nf}')

Rank   0
Movie   0
ReleaseDate   0
Revenues   0
Tickets_Sold   0
Legs   0
Budget   0
WBO_to_Budget_Ratio   0
Theater_Count   0
Average_Run   0
Genre   0
Source   0
Creative_Type   0
Distributor   0


In [360]:
for col_name in col_reorder:
    count_nan = movies_final_2019_df[col_name].isna().sum()
    print(f'{col_name}   {count_nan}')



Rank   0
Movie   0
ReleaseDate   0
Revenues   0
Tickets_Sold   0
Legs   0
Budget   485
WBO_to_Budget_Ratio   485
Theater_Count   37
Average_Run   37
Genre   0
Source   19
Creative_Type   13
Distributor   0


In [364]:
#Pickle final file
# movies_final_2019_df.to_pickle('final_2019_pk')